In [1]:
import pandas as pd
import itertools

In [10]:
file = 'legacy_data/Amtsblatt_1918.xlsx'
res_type_scheme, _ = SkosConceptScheme.objects.get_or_create(dc_title='res_type')
archiv, _ = Institution.objects.get_or_create(
    written_name='Wiener Stadt- und Landesarchiv',
    abbreviation="WStLA",
    institution_type="Archiv"
)

In [11]:
df = pd.read_excel(file).fillna('False')

In [12]:
df

,Teil von Bestand,(Archiv)Signatur,Punkt; Seite,Titel,Typ,Amtsblatt Nr. u. Datum,Datum original,Datum normalisiert DD/MM/YYYY),Personen,Institutionen,Orte,Schlagwörter normalisiert,Verweis,Zusammenfassung,Kommentar
0,"Wiener Stadt- und Landesarchiv, Archivbiblioth...",M 599 27/4 2. Ex.,o.P.; 1996,Stadtrat. Bericht über die Sitzung des Stadtra...,Amtsblatteintrag,"Nr. 82, 11. Oktober 1918, Jg. 27",1918-10-04,1918-10-04,Richard Weiskirchner; Franz Spalowsky,False,Wien,Lohnforderung; Approvisierung; Teuerung; Zulag...,GR-Sitzung im August 1918,Anfrage des StR Spalowsky bzgl. Lohnforderunge...,False
1,"Wiener Stadt- und Landesarchiv, Archivbiblioth...",M 599 27/4 2. Ex.,o.P.; 2051,Stadtrat. Bericht über die Sitzung des Stadtra...,Amtsblatteintrag,"Nr. 84, 18. Oktober 1918, Jg. 27",1918-10-10,1910-10-10,Franz Hoß; Franz Spalowsky; Josef Müller; Oswa...,False,WIen,Angestellte; Kriegszulage,"GR-Sitzung, 15.10.1918",Bericht des VBM Hoß über Neuregelung der Krieg...,False
2,"Wiener Stadt- und Landesarchiv, Archivbiblioth...",M 599 27/4 2. Ex.,31; 2072-2091,Gemeinderat. Stenographischer Bericht über die...,Amtsblatteintrag,"Nr., 85, 22. Oktober 1918, 17.Jg.",1918-10-15,1918-10-15,Richard Weiskirchner; Franz Hoß; Oskar Hein; H...,Straßenbahnen (städtische),Wien,Zulage; Lohnforderung; Lehrer; Streik; Unterne...,"StR-Sitzung, 10.10.1918","31. (Z. 9988, Post 15) Referent VB Hoß: Neureg...",False
3,"Wiener Stadt- und Landesarchiv, Archivbiblioth...",M 599 27/4 2. Ex.,15; 2116,Gemeinderat. Stenographischer Bericht über die...,Amtsblatteintrag,"Nr. 87, 29. Oktober 1918",1918-10-25,1918-10-25,Franz Hoß; Richard Weikirchner; Georg Philip,False,Wien,Dienstordnung; Kanzleiorgan; Zulage; Hierarchien,False,"16. (Z.9941, Post 1). Referent VBM Hoß: betrif...",False
4,"Wiener Stadt- und Landesarchiv, Archivbiblioth...",M 599 27/4 2. Ex.,o.P.; 2180,Stadtrat. Bericht über die Sotzung des Stadtra...,Amtsblatteintrag,"Nr. 90, 8. November 1918",1918-10-30,1918-10-30,False,Straßenbahnen (städtische);,Wien,Straßenbahnbedienstete; Zulage; Lohnverhandlun...,False,"(P.Z.10761, St.Str.B.,3501) Allgemeine Zuwendu...",False
5,"Wiener Stadt- und Landesarchiv, Archivbiblioth...",M 599 27/4 2. Ex.,10; 2215,Gemeinderat. Stenographischer Bericht über die...,Amtsblatteintrag,"Nr. 93, 19.November 1918",1918-11-13,1918-11-13,Richard Weiskirchner; Jakob Reumann; Karl Lueger,False,Wien,Koalitionsfreiheit; Lehrer; Gelöbnis,False,10. Interpellation von Jakob Reumann ob nun da...,False
6,"Wiener Stadt- und Landesarchiv, Archivbiblioth...",M 599 27/4 2. Ex.,16; 2218-2224,Gemeinderat. Stenographischer Bericht über die...,Amtsblatteintrag,"Nr. 93, 19.November 1918",1918-11-13,1918-11-13,Franz Hoß; Heinrich Hierhammer; Franz Spalowsky,Magistrat der Stadt Wien; Staatsangestelltenau...,Wien,Zulage; Lehrer; Angestellte; Dienstzeitanrechn...,GR-Beschluss vom 15.10.1918,"16. (Z. 11001, Post 20) Referent VBM Hoß. Ergä...",False
7,"Wiener Stadt- und Landesarchiv, Archivbiblioth...",M 599 27/4 2. Ex.,17; 2224f.,Gemeinderat. Stenographischer Bericht über die...,Amtsblatteintrag,"Nr. 93, 19.November 1918",1918-11-13,1918-11-13,Franz Hoß; Heinrich Hierhammer,Gaswerke (städtische),Wien,Lohnregelung; Personalkosten; Zulage; Teuerung...,False,"17. (Z.11147, Post 27) Referenz VBM Hoß: Lohnr...",False
8,"Wiener Stadt- und Landesarchiv, Archivbiblioth...",M 599 27/4 2. Ex.,20; 2225f.,Gemeinderat. Stenographischer Bericht über die...,Amtsblatteintrag,"Nr. 93, 19.November 1918",1918-11-13,1918-11-13,Karl Angermayer; Heinrich Hierhammer,Fuhrwerksbetriebe (städtische);,Wien,Lohnregelung; Dienstverhältnis; Arbeiter; Lohn...,False,"20. (Z.11046, Post 22) Referent GR Angermayer:...",False
9,"Wiener Stadt- und Landesarchiv, Archivbiblioth...",M 599 27/4 2. Ex.,30; 2228f.,Gemeinderat. Stenographischer Bericht über die...,Amtsblatteintrag,"Nr. 93, 19.November 1918",1918-11-13,1918-11-13,Franz Spalowsky; Ferdinand Skaret; Josef Rain,Organisationen der Straßenbahnbediensteten; St...,Wien,Unternehmungen; Bedien

In [13]:
for i, row in df.iterrows():
    item = None
    signatur = ", ".join(
        [
            "Bestand: {}".format(str(row.get('Teil von Bestand', default = "-"))), 
            "Signatur: {}".format(str(row.get('(Archiv)Signatur', default='-'))), 
            "Punkt/Seite: {}".format(str(row.get('Punkt; Seite', default='-'))), 
            "[internal-id]: {}".format(i)
        ]
    )
#     print(signatur)
    try:
        item, _ = ArchResource.objects.get_or_create(
            signature=signatur
        )
    except Exception as e:
        print(e)
    if item:
        if row['Schlagwörter normalisiert'] != 'False':
            slw = row['Schlagwörter normalisiert'].split(';')
            concepts = []
            for y in slw:
                concepts.append(list(SkosConcept.objects.filter(pref_label=y.strip())))
                concepts.append(list(SkosConcept.objects.filter(other_label__label=y.strip())))
            if concepts:
                item.subject_norm.set(list(itertools.chain.from_iterable(concepts)))
        if row['Titel'] != 'False':
            item.title = row['Titel']
        if row['Zusammenfassung'] != 'False':
            item.abstract = row['Zusammenfassung']
        if row['Kommentar'] != 'False':
            item.notes = row['Kommentar']
#         if row['Datum original'] != 'False':
#             item.written_date = row['Datum original'].replace('00:00:00', '')
        if row['Datum normalisiert DD/MM/YYYY)'] != 'False':
            try:
                item.not_before = pd.to_datetime(row['Datum normalisiert DD/MM/YYYY)'])
            except Exception as e:
                print(e)
        if row['Typ'] != 'False':
            type_vocab, _ = SkosConcept.objects.get_or_create(
                pref_label="{}".format(row['Typ'])
            )
            type_vocab.scheme.add(res_type_scheme)
            item.res_type = type_vocab
        if row['Orte'] != 'False':
            for x in row['Orte'].split(';'):
                try:
                    pl = Place.objects.get(name=x.strip())
                except Exception as e:
                    pl = None
                    print(e, x.strip())
                if pl:
                    item.mentioned_place.add(pl)
        if row['Institutionen'] != 'False':
            for x in row['Institutionen'].split(';'):
                try:
                    pl = Institution.objects.get(written_name=x.strip())
                except Exception as e:
                    pl = None
                    print(e, x.strip())
                if pl:
                    item.mentioned_inst.add(pl)
        if row['Personen'] != 'False':
            for x in row['Personen'].split(';'):
                try:
                    pl = Person.objects.get(written_name=x.strip())
                except Exception as e:
                    pl = None
                    print(e, x.strip())
                if pl:
                    item.mentioned_person.add(pl)
        item.archiv = archiv
        item.save()

Person matching query does not exist. Richard Weikirchner
Institution matching query does not exist. 
Person matching query does not exist. Karl Lueger
Institution matching query does not exist. Staatsangestelltenausschuss des Abgeordnetenhauses
Institution matching query does not exist. Gaswerke (städtische)
Institution matching query does not exist. 
Institution matching query does not exist. Organisationen der Straßenbahnbediensteten
Person matching query does not exist. Karl Rummenlhardt
Person matching query does not exist. Ludwig Spängler
Person matching query does not exist. Hornek
Institution matching query does not exist. Gaswerke (städtische)
Institution matching query does not exist. Gemeinderats-Ausschuss zur Beratung einer Neuordnung der Gemeindeverfassung und des Gemeindewahlrechts
Institution matching query does not exist. Gemeinderats-Ausschuss zur Beratung einer Neuordnung der Gemeindeverfassung und des Gemeindewahlrechts
Person matching query does not exist. Dr. Hilde

In [ ]:
# Person.objects.all().delete()

In [ ]:
# ArchResource.objects.all().delete()